# Overview:

## Broadband Sensitivity Calculations:
1. Import Excel Spreadsheet.py
2. Call calcByAngle and store in a variable to create a function from observation zenith angle to various outputs
3. Call created function with requested angle from zenith. *This is 90 - elevation angle*
4. Create output display functions if necessary
5. Call requested results

In [ ]:
import sensitivity_calculator as sens
import numpy as np

inputs = {'diameter': 5.7, 't': 273, 'wfe': 10.7, 'eta': 0.98, 'doe': 0.8, 't_int': 1, 'pixelYield': 0.8, 
        'szCamNumPoln': 1, 'eorSpecNumPoln': 2, 't_filter_cold': np.array([0.593, 0.84 , 0.909, 0.922, 0.89 ]), 
        't_lens_cold': np.array([0.98, 0.98, 0.98, 0.98, 0.98]), 
        't_uhdpe_window': np.array([0.96, 0.97, 0.97, 0.97, 0.97]), 
        'singleModedAOmegaLambda2': np.array([1., 1., 1., 1., 1.]), 
        'spatialPixels': np.array([36450, 20808, 10368, 10368,  7938]), 
        'eqbw': np.array([9.7e+10, 3.0e+10, 3.6e+10, 7.0e+10, 5.6e+10]), 
        'centerFrequency': np.array([8.50e+11, 4.05e+11, 3.48e+11, 2.85e+11, 2.22e+11]), 'detectorNEP': 0, 
        'backgroundSubtractionDegradationFactor': 1, 'r': np.array([[100., 100., 100.],
                                            [100., 100., 100.],
                                            [100., 100., 100.],
                                            [100., 100., 100.],
                                            [100., 100., 100.]]), 
        'decimalPlaces': 3, 'observationElevationAngle': 45, 'outputFreq': True, 
        'detectorSpacing': np.array([1.51, 1.97, 2.75, 2.75, 3.13]), 'lyotStopAngle': 13.4}
# Alternatively, sens.getInputs("input.yaml")

spillEfficiency = sens.getSpillEfficiency(inputs)
calculate = sens.calcByAngle(inputs["diameter"], inputs["t"], inputs["wfe"], inputs["eta"], inputs["doe"], inputs["t_int"], inputs["pixelYield"], inputs["szCamNumPoln"], inputs["eorSpecNumPoln"],
                            inputs["t_filter_cold"], inputs["t_lens_cold"], inputs["t_uhdpe_window"], spillEfficiency, inputs["singleModedAOmegaLambda2"],
                            inputs["spatialPixels"], inputs["eqbw"], inputs["centerFrequency"], inputs["detectorNEP"],
                            inputs["backgroundSubtractionDegradationFactor"], inputs["r"])
outputs = calculate(45)

print(outputs["netW8Avg"])

ell, N_ell_T_full, N_ell_P_full = sens.getNoiseCurves(inputs, outputs)

## EoR Calculations
1. Import sensitivity calculator
2. Create dictonary of parameters specifying detector
3. Create list of pairs of finesse and frequency values to get noise curves for
4. Call sens.eorNoiseCurves
5. Index specific noise curve from returned dictonary

In [ ]:

import sensitivity_calculator as sens
import numpy as np

inputs = {'diameter': 5.7, 't': 273, 'wfe': 10.7, 'eta': 0.98, 'doe': 0.8, 'pixelYield': 0.8,
              'eorSpecNumPoln': 2, 't_filter_cold': np.array([1, 1]), 't_lens_cold': np.array([.98, .98]), 't_uhdpe_window': np.array([1, 1]), 'spatialPixels': np.array([3456, 3072]),
              'centerFrequency': np.array([262.5*10**9, 367.5*10**9]), 'detectorNEP': 0,
              'backgroundSubtractionDegradationFactor': 1, 'observationElevationAngle': 45, 'detectorSpacing': np.array([2.75, 2.09]), 'lyotStopAngle': 13.4}
# diameter is the diameter of the telescope in m
# t is the temperature of the telescope in K
# wfe is wave front error in um
# eta is telescope efficiency
# doe is detector quantum efficiency; loss due to superconducting detectors
# pixelYield is the proportion of working pixels
# eorSpecNumPoln is the number of polarization modes for EoR spectrometers
# t_filter_cold, t_lens_cold, and t_uhdpe_window all characterize the throughput of the instruments. However, we don't have experimental values for these yet, so they currently are an estimation.
# spatialPixels is the total number of pixels on all arrays
# centerFrequency is the center of the frequency range for the detector arrays
# detectorNEP is the NEP from the detector itself
# backgroundSubtractionDegradationFactor is an efficiency loss factor from post analysis removing of common atmospheric noise
# observationElevationAngle is the angle above the horizon that the telescope is observing
# detectorSpacing is the distance between pixels in the arrays in mm
# lyotStopAngle is the angle from the detectors to the lyot stop, which controls the spill efficiency

rfpairs = np.array([(100, 250*10**9), (150, 350*10**9), (1000, 400*10**9)])
# rfpairs is a list of (resolving power, center frequency) to get noise curves for
# Note that center frequency is in Hz

allCurves = sens.eorNoiseCurves(inputs, rfpairs)
ell_250, N_ell_T_full_250, N_ell_P_full_250 = allCurves[(100, 250*10**9)]
ell_400, N_ell_T_full_400, N_ell_P_full_400 = allCurves[(1000, 400*10**9)]

## Map Simulations
1. Run broadband calculations.
2. Call ccat_mapsims with input parameters, output from broadband calculations, band and tube of detector, pysm components (if wanted), seed (if wanted), whether to simulate the cmb, and whether to simulate noise.
3. Feed to healpy's anafast to analyze the power spectrum.

In [ ]:
import sensitivity_calculator as sens
import healpy as hp

seed = 0

# Replace path with path on local machine
zf = sens.zeroHitmapFraction(
    "/home/amm487/cloned_repos/Sensitivity-Calculator/src/sensitivity_calculator/data/ccat_uniform_coverage_nside256_201021.fits", 256)

ccat280map = sens.ccat_mapsims(
    inputs, outputs, "HF2", "LC1", "d1", seed, sim_cmb=False, sim_noise=True)
ccat280cls = hp.sphtfunc.anafast(ccat280map)
sens.plotPowerSpectrum(ccat280cls[0], ccat280cls[1], ccat280cls[2],
                    "280 GHz CCAT Noise", zf=zf)
so280map = sens.so_mapsims("UHF2", "LT0", "d1",
                    seed, sim_cmb=False, sim_noise=True)
so280cls = hp.sphtfunc.anafast(so280map)
sens.plotPowerSpectrum(so280cls[0], so280cls[1], so280cls[2],
                    "280 GHz SO Noise", zf=zf)
ccat850map = sens.ccat_mapsims(
    inputs, outputs, "HF5", "LC3", "d1", seed, sim_cmb=False, sim_noise=True)
ccat850cls = hp.sphtfunc.anafast(ccat850map)
sens.plotPowerSpectrum(ccat850cls[0], ccat850cls[1], ccat850cls[2],
                    "850 GHz CCAT Noise", zf=zf)